In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import power

In [ ]:
def calculate_avg_acc_rewards(run_acc_rewards, num_episodes, num_runs):
    """
    Function to calculate the average accumulated rewards
    """
    avg_acc_rewards = []
    for i in range(num_episodes):
        avg_acc_reward = 0
        for j in range(num_runs):
            avg_acc_reward += run_acc_rewards[j][i]
        avg_acc_reward /= num_runs
        avg_acc_rewards.append(avg_acc_reward)

    return avg_acc_rewards

def plot_avg_acc_rewards(avg_acc_rewards, title):
    """
    Function to plot the average accumulated rewards
    """
    plt.figure(figsize=(10, 6))

    for k, v in avg_acc_rewards.items():
        plt.plot(v)

    plt.xlabel("Episodes")
    plt.ylabel("Average Accumulated Rewards")
    plt.title(title)
    plt.legend(avg_acc_rewards.keys())
    plt.show()

### Question 1

#### Defining Basic Functions

In [ ]:
GOAL_STATE = (3, 13)
START_STATE = (15, 4) 

def init_env() -> np.ndarray:
    """
    Function to initialize the environment matrix
    """
    # Environment matrix
    # regular = 0    wall = 1    oil = 2     bump = 3    start = 4    end = 5
    Env = np.zeros([20, 20])

    # Defining border walls
    Env[:, [0]] = np.ones([20, 1])
    Env[:, [19]] = np.ones([20, 1])
    Env[0, :] = np.ones([1, 20])
    Env[19, :] = np.ones([1, 20])

    # Defining cell properties inside the maze
    # wall = 1
    Env[2, 5] = 1
    Env[3, 5] = 1
    Env[4, 3:17] = np.ones([1, 14])
    Env[5, 3] = 1
    Env[6, 3] = 1; Env[6, 6] = 1; Env[6, 9] = 1; Env[6, 15] = 1
    Env[7, 3] = 1; Env[7, 6] = 1; Env[7, 9] = 1; Env[7, 12:16] = np.ones([1, 4])
    Env[8, 6] = 1; Env[8, 9] = 1; Env[8, 15] = 1
    Env[9, 6] = 1; Env[9, 9] = 1; Env[9, 15] = 1
    Env[10, 1:5] = np.ones([1, 4]); Env[10, 6] = 1; Env[10, 9:11] = [1, 1]; Env[10, 15] = 1
    Env[11, 6] = 1; Env[11, 10] = 1; Env[11, 13] = 1; Env[11, 15:18] = np.ones([1, 3])
    Env[12, 3:8] = np.ones([1, 5]); Env[12, 10] = 1; Env[12, 13] = 1; Env[12, 17] = 1
    Env[13, 7] = 1; Env[13, 10] = 1; Env[13, 13] = 1; Env[13, 17] = 1
    Env[14, 7] = 1; Env[14, 10] = 1; Env[14, 13] = 1
    Env[15, 7] = 1; Env[15, 13:17] = [1, 1, 1, 1]
    Env[17, 1:3] = [1, 1]; Env[17, 7:13] = [1, 1, 1, 1, 1, 1]

    # oil = 2
    Env[2, 8] = 2; Env[2, 16] = 2
    Env[4, 2] = 2
    Env[5, 6] = 2
    Env[10, 18] = 2
    Env[14, 14] = 2
    Env[15, 10] = 2
    Env[16, 10] = 2
    Env[17, 14] = 2; Env[17, 17] = 2
    Env[18, 7] = 2

    # bump = 3
    Env[1, 11] = 3; Env[1, 12] = 3
    Env[2, 1:4] = [3, 3, 3]
    Env[5, 1] = 3; Env[5, 9] = 3; Env[5, 17] = 3
    Env[6, 17] = 3
    Env[7, 2] = 3; Env[7, 10:12] = [3, 3]; Env[7, 17] = 3
    Env[8, 17] = 3
    Env[12, 11:13] = [3, 3]
    Env[14, 1:3] = [3, 3]
    Env[15, 17:19] = [3, 3]
    Env[16, 7] = 3

    # start = 4
    Env[15, 4] = 4

    # goal = 5
    Env[GOAL_STATE[0], GOAL_STATE[1]] = 5

    return Env

def plot_env(Env, title, annot_matrix=False, show=True) -> None:
    """
    Function to plot the environment matrix
    """
    # Define colors
    colors = {
        0: [1, 1, 1],        # White
        1: [0, 0, 0],        # Black
        2: [0.55, 0, 0],     # Light Brown
        3: [0.96, 0.8, 0.6], # Dark Red
        4: [0, 0, 1],        # Green
        5: [0, 1, 0]         # Blue
    }


    rgb_maze = np.zeros((Env.shape[0], Env.shape[1], 3))
    for i in range(Env.shape[0]):
        for j in range(Env.shape[1]):
            rgb_maze[i, j] = colors.get(Env[i, j], [1, 1, 1])

    if annot_matrix is not False:
        annot_matrix = np.where(Env == 1, '', annot_matrix)

    plt.figure(figsize=(15, 10))
    sns.heatmap(Env,fmt="",  cmap=sns.color_palette([colors[i] for i in range(6)]), cbar=False,annot=annot_matrix, linewidths=0.5, linecolor='black')
    plt.axis('off')
    plt.title(title)

    if show:
        plt.show()

def plot_policy(Env, policy, title, curr_state = None) -> None:
    """
    Function to plot the optimal policy matrix
    """
    plot_env(Env, title, annot_matrix=False, show=False)
    for i in range(20):
        for j in range(20):
            if policy[i, j] == "up":
                plt.arrow(j+0.5, i+0.85, 0, -0.5, width=0.04, color='black')  # Up
            if policy[i, j] == "right":
                plt.arrow(j+0.15, i+0.5, 0.5, 0, width=0.04, color='black')  # Right
            if policy[i, j] == "down":
                plt.arrow(j+0.5, i+0.15, 0, 0.50, width=0.04, color='black')  # Down
            if policy[i, j] == "left":
                plt.arrow(j+0.85, i+0.5, -0.5, 0, width=0.04, color='black')  # Left

    if curr_state:
        circle = plt.Circle((curr_state[1] + 0.5, curr_state[0] + 0.5), 0.3, color='blue', fill=True, label='Current State')
        plt.gca().add_artist(circle)

    plt.show()

def plot_optimal_path(Env, path, title) -> None:
    """
    Function to plot the optimal path
    """
    plot_env(Env, title, annot_matrix=False, show=False)
    
    for state_cr, direction in path:
        r = state_cr[0] # x_coordinate
        c = state_cr[1] # y_coordinate

        if direction == 'right':
            plt.arrow(c + 0.5, r + 0.5, 0.8, 0, width=0.04, color='black')   # Right
        if direction == 'left':
            plt.arrow(c + 0.5, r + 0.5, -0.8, 0, width=0.04, color='black')  # Left
        if direction == 'up':
            plt.arrow(c + 0.5, r + 0.5, 0, -0.8, width=0.04, color='black')  # Up
        if direction == 'down':
            plt.arrow(c + 0.5, r + 0.5, 0, 0.8, width=0.04, color='black')  # Down

    plt.show()

def get_reward(next_state_type, wall_hit) -> float:
    """
    Function to return the reward of an action based on next state and if wall was hit
    """
    next_state_type = int(next_state_type)
    reward = -1 # For taking an action

    if wall_hit:         # Wall
        reward += -0.8

    if next_state_type == 2:  # Oil
        reward += -5
    elif next_state_type == 3:  # Bump
        reward += -10
    elif next_state_type == 5:  # Goal
        reward += 300
    
    return reward

def get_next_state(Env, state, action, p = None, deterministic = False) -> tuple:
    """
    Function to return the next state given the current state and action
    """
    i, j = state[0], state[1]
    wall_hit = False
    
    if not deterministic:
        probabilities = get_probabilities(action, p)
        action = np.random.choice(['up', 'down', 'left', 'right'], p=[probabilities['up'], probabilities['down'], probabilities['left'], probabilities['right']])

    if action == 'up':
        next_i, next_j = i - 1, j
    elif action == 'down':
        next_i, next_j = i + 1, j
    elif action == 'left':
        next_i, next_j = i, j - 1
    elif action == 'right':
        next_i, next_j = i, j + 1

    if Env[next_i, next_j] == 1:  # Wall
        next_i, next_j = i, j
        wall_hit = True
    
    return next_i, next_j, wall_hit

def get_probabilities(action, p) -> list:
    """
    Function to return the probabilities of each action given the current action
    """
    probabilities = {"up": 0, "down": 0, "left": 0, "right": 0}
    if action == "up":
        probabilities["up"] = 1 - p
        probabilities["down"] = 0
        probabilities["left"] = p / 2
        probabilities["right"] = p / 2
    elif action == "down":
        probabilities["up"] = 0
        probabilities["down"] = 1 - p
        probabilities["left"] = p / 2
        probabilities["right"] = p / 2
    elif action == "left":
        probabilities["up"] = p / 2
        probabilities["down"] = p / 2
        probabilities["left"] = 1 - p
        probabilities["right"] = 0
    elif action == "right":
        probabilities["up"] = p / 2
        probabilities["down"] = p / 2
        probabilities["left"] = 0
        probabilities["right"] = 1 - p
    
    return probabilities

def get_optimal_path(Env, start, end, optimal_policy):
    """
    Function to get the optimal path from start to end
    """
    curr_state = start
    path = []
    visited_states = []
    optimal_path_found = True
    while curr_state != end:
        if curr_state in visited_states:
            optimal_path_found = False
            print("No optimal path found.")
            return path, optimal_path_found

        visited_states.append(curr_state)
        i, j = curr_state[0], curr_state[1]
        action = optimal_policy[i, j]
        path.append((curr_state, action))
        next_i, next_j, wall_hit = get_next_state(Env, curr_state, action, deterministic = True)
        curr_state = (next_i, next_j)

    return path, optimal_path_found

def get_random_state(Env):
    """
    Function to get a random start state
    """
    while True:
        i = np.random.randint(1, 19)
        j = np.random.randint(1, 19)

        if Env[i, j] != 1 and Env[i, j] != 5:  # should not be the goal state or a wall
            break
    return (i, j)

def epsilon_greedy(Q, state, epsilon):
    """
    Function to return the action based on epsilon greedy policy
    """
    possible_actions = ['up', 'down', 'left', 'right']
    if np.random.uniform(0, 1) < epsilon:
        action = np.random.choice(possible_actions)
    else:
        i, j = state[0], state[1]
        action_index = np.argmax(Q[i, j])
        action = possible_actions[action_index]

    return action

def get_preferences(H, state):
    """
    Function to return the preferences of each action given H and a state.
    """
    logits = H[state[0], state[1]]
    max_logit = np.max(logits)  # for stability
    exp_logits = np.exp(logits - max_logit)
    preferences = exp_logits / np.sum(exp_logits)
    return preferences


#### Functions for TD algorithms

In [ ]:
def q_learning(Env, p, gamma, alpha, epsilon, num_episodes, max_episode_length):
    """
    Function to perform Q-Learning
    """
    Q = np.zeros((Env.shape[0], Env.shape[1], 4))
    policy = np.full((Env.shape[0], Env.shape[1]), '', dtype=object)
    possible_actions = ['up', 'down', 'left', 'right']
    acc_rewards = [0]

    # Repeating for each episode
    for _ in range(num_episodes):
        curr_state = get_random_state(Env)
        episode_length = 0
        ep_acc_reward = 0

        # Repeating for each step of the episode
        while episode_length < max_episode_length:
            i, j = curr_state[0], curr_state[1]
            action = epsilon_greedy(Q, curr_state, epsilon)

            next_i, next_j, wall_hit = get_next_state(Env, curr_state, action, p)
            reward = get_reward(Env[next_i, next_j], wall_hit)
            ep_acc_reward += reward
            next_state = (next_i, next_j)
            curr_Q = Q[i, j, possible_actions.index(action)]

            Q[i, j, possible_actions.index(action)] = curr_Q + alpha * (reward + gamma * np.max(Q[next_i, next_j]) - curr_Q)
            policy[i, j] = possible_actions[np.argmax(Q[i, j])]

            curr_state = next_state
            episode_length += 1
        
        avg_ep_acc_reward = ep_acc_reward / episode_length
        acc_rewards.append(avg_ep_acc_reward)

    return policy, Q, acc_rewards

def sarsa(Env, p, gamma, alpha, epsilon, num_episodes, max_episode_length):
    """
    Function to perform SARSA
    """
    Q = np.zeros((Env.shape[0], Env.shape[1], 4))
    policy = np.full((Env.shape[0], Env.shape[1]), '', dtype=object)
    possible_actions = ['up', 'down', 'left', 'right']
    acc_rewards = [0]

    # Repeating for each episode
    for _ in range(num_episodes):
        curr_state = get_random_state(Env)
        curr_action = epsilon_greedy(Q, curr_state, epsilon)
        episode_length = 0
        ep_acc_reward = 0

        # Repeating for each step of the episode
        while episode_length < max_episode_length:
            i, j = curr_state[0], curr_state[1]

            next_i, next_j, wall_hit = get_next_state(Env, curr_state, curr_action, p)
            reward = get_reward(Env[next_i, next_j], wall_hit)

            curr_Q = Q[i, j, possible_actions.index(curr_action)]
            ep_acc_reward += reward

            next_state = (next_i, next_j)
            next_action = epsilon_greedy(Q, next_state, epsilon)
            next_Q = Q[next_i, next_j, possible_actions.index(next_action)]

            Q[i, j, possible_actions.index(curr_action)] = curr_Q + alpha * (reward + gamma * next_Q - curr_Q)
            policy[i, j] = possible_actions[np.argmax(Q[i, j])]

            curr_state = next_state
            curr_action = next_action
            episode_length += 1
        
        avg_ep_acc_reward = ep_acc_reward / episode_length
        acc_rewards.append(avg_ep_acc_reward)

    return policy, Q, acc_rewards

def actor_critic(Env, p, gamma, alpha, beta, num_episodes, max_episode_length):
    """
    Refactored Actor-Critic algorithm with stable softmax, full gradient updates, and better initialization.
    """
    rows, cols = Env.shape
    V = np.random.normal(0, 0.01, (rows, cols))
    H = np.random.normal(0, 0.01, (rows, cols, 4))
    policy = np.full((rows, cols), '', dtype=object)
    possible_actions = ['up', 'down', 'left', 'right']
    acc_rewards = []

    for ep in range(num_episodes):
        curr_state = get_random_state(Env)
        episode_length = 0
        ep_acc_reward = 0

        while episode_length < max_episode_length:
            i, j = curr_state
            preferences = get_preferences(H, curr_state)

            curr_action_idx = np.random.choice(range(4), p=preferences)
            curr_action = possible_actions[curr_action_idx]

            next_i, next_j, wall_hit = get_next_state(Env, curr_state, curr_action, p)
            reward = get_reward(Env[next_i, next_j], wall_hit)
            ep_acc_reward += reward
            next_state = (next_i, next_j)

            delta = reward + gamma * V[next_i, next_j] - V[i, j]
            V[i, j] += alpha * delta
            H[i, j, curr_action_idx] += beta * delta * (1 - preferences[curr_action_idx])

            policy[i, j] = possible_actions[np.argmax(H[i, j])]

            curr_state = next_state
            episode_length += 1

        avg_ep_reward = ep_acc_reward / episode_length if episode_length > 0 else 0
        acc_rewards.append(avg_ep_reward)

    return policy, H, acc_rewards

def simulate(algo_name, num_runs, p, gamma, alpha = 0.01, beta = 0.05, epsilon = 0.1, num_episodes = 5000, max_episode_length = 200):
    """
    Function to simulate TD algorithms for multiple runs
    """
    optimal_path_found_counts = 0
    run_acc_rewards = []
    for i in range(num_runs):
        Env = init_env()
        print(f"\nRun {i+1}")

        if algo_name == "Q-Learning":
            policy, Q, acc_rewards = q_learning(Env, p, gamma, alpha, epsilon, num_episodes, max_episode_length)

        elif algo_name == "SARSA":
            policy, Q, acc_rewards = sarsa(Env, p, gamma, alpha, epsilon, num_episodes, max_episode_length)
        
        elif algo_name == "Actor_Critic":
            policy, H, acc_rewards = actor_critic(Env, p, gamma, alpha, beta, num_episodes, max_episode_length)

        run_acc_rewards.append(acc_rewards)
        
        plot_policy(Env, policy, f"{algo_name} Optimal Policy - Run {i+1}")

        optimal_path, optimal_path_found = get_optimal_path(Env, START_STATE, GOAL_STATE, policy)
        plot_optimal_path(Env, optimal_path, f"{algo_name} Optimal Path - Run {i+1}")

        if optimal_path_found:
            optimal_path_found_counts += 1

    print(f"Optiaml Path Found: {optimal_path_found_counts}\n")

    avg_acc_rewards = calculate_avg_acc_rewards(run_acc_rewards, num_episodes, num_runs)
    return avg_acc_rewards

def simulate_all(num_runs, p, gamma, alpha, beta, epsilon, num_episodes, max_episode_length):
    algos = ["Actor_Critic", "Q-Learning", "SARSA"]
    avg_acc_rewards = {algo: None for algo in algos}
    for algo in algos:
        print(algo)
        if algo == "Actor_Critic":
            avg_acc_rewards[algo] = simulate(algo, num_runs, p, gamma)
        else:
            avg_acc_rewards[algo] = simulate(algo, num_runs, p, gamma, alpha, beta, epsilon, num_episodes, max_episode_length)
    
    plot_avg_acc_rewards(avg_acc_rewards, f"Average Accumulated Rewards")
    
    return avg_acc_rewards

In [ ]:
p = 0.025
gamma = 0.96
alpha = 0.25
beta = 0.05
epsilon = 0.1
max_episode_length = 1000
num_episodes = 1000
num_runs = 10

avg_acc_rewards = simulate_all(num_runs, p, gamma, alpha, beta, epsilon, num_episodes, max_episode_length)

In [ ]:
avg_acc_rewards_new = {
    "Q-Learning":avg_acc_rewards["Q-Learning"],
    "SARSA": avg_acc_rewards["SARSA"]
}
plot_avg_acc_rewards(avg_acc_rewards_new, f"Average Accumulated Rewards")


In [ ]:
avg_acc_rewards_new = {
    "Actor_Critic": avg_acc_rewards["Actor_Critic"],
}
plot_avg_acc_rewards(avg_acc_rewards_new, f"Average Accumulated Rewards")


### Question 2

In [ ]:
# Define parameters
NUM_STATES = 16
NUM_ACTIONS = 4
NUM_GENES = 4
ACTIONS = {'a1': [0, 0, 0, 0], 'a2': [0, 1, 0, 0], 'a3': [0, 0, 1, 0], 'a4': [0, 0, 0, 1]}
STATES = {'s1': [0, 0, 0, 0], 's2': [0, 0, 0, 1], 's3': [0, 0, 1, 0], 's4': [0, 0, 1, 1], 's5': [0, 1, 0, 0], 's6': [0, 1, 0, 1], 's7': [0, 1, 1, 0], 's8': [0, 1, 1, 1], 's9': [1, 0, 0, 0], 's10': [1, 0, 0, 1], 's11': [1, 0, 1, 0], 's12': [1, 0, 1, 1], 's13': [1, 1, 0, 0], 's14': [1, 1, 0, 1], 's15': [1, 1, 1, 0], 's16': [1, 1, 1, 1]}
C = np.array([[0, 0, -1, 0],
              [1, 0, -1, -1],
              [0, 1, 0, 0],
              [-1, 1, 1, 0]])


def xor_mod2(vec1, vec2):
    """Performs component-wise XOR (mod 2 addition)."""
    return (vec1 + vec2) % 2

def v_bar(vec):
    output = np.zeros(len(vec))
    for i, elem in enumerate(vec):
        if elem > 0:
            output[i] = 1
        else:
            output[i] = 0
    
    return output

def compute_N(p):
    """Returns a list of length NUM_GENES, where each element is 1 with probability p and 0 with probability 1-p."""
    N = []
    for _ in range(NUM_GENES):
        N.append(int(np.random.rand() < p))
    return N

def get_random_state_idx():
    """Returns a random state index."""
    random_idx = np.random.randint(0, NUM_STATES)
    return random_idx

def compute_next_state_idx(p, state_idx, action_idx):
    """Computes the next state index for a given action index and state index."""
    N = compute_N(p)
    action = ACTIONS[f'a{action_idx+1}']
    state = STATES[f"s{state_idx+1}"]
    next_state = list(xor_mod2(xor_mod2(v_bar(C @ state), action), N))
    next_state_idx = list(STATES.values()).index(next_state)

    return next_state_idx

def action_cost(action_idx):
    """Computes the cost of an action."""
    if action_idx == "a1" or action_idx == "a4":
        return 0
    
    return 1

def init_reward_function():
    """Defines the reward function."""
    R = {}
    for action in list(ACTIONS.keys()):
        R[action] = np.zeros((NUM_STATES, NUM_STATES))

        for i in range(NUM_STATES):
            for j in range(NUM_STATES):
                next_state = STATES[f"s{j+1}"]
                R[action][i, j] = 5 * sum(next_state) - action_cost(action)

    return R

def epsilon_greedy(Q, state_idx, epsilon):
    """
    Function to return the action based on epsilon greedy policy
    """
    if np.random.uniform(0, 1) < epsilon:
        action_index = np.random.randint(0, NUM_ACTIONS)
    else:
        action_index = np.argmax(Q[state_idx])

    return action_index

def get_preference(H, curr_state_idx, action_idx = None):
    """
    Function to return the preferences of each action given H and a state.
    """
    sigma_e = 0

    for i in range(NUM_ACTIONS):
        sigma_e += power(np.e, H[curr_state_idx, i])

    if action_idx is None:
        preferences = np.zeros(NUM_ACTIONS)
        for i in range(NUM_ACTIONS):
            preferences[i] = power(np.e, H[curr_state_idx, i]) / sigma_e

        return preferences

    preference = power(np.e, H[curr_state_idx, action_idx]) / sigma_e
    return preference


In [ ]:
def q_learning(p, gamma, alpha, epsilon, num_episodes, max_episode_length):
    """
    Function to perform Q-Learning
    """
    Q = np.zeros((NUM_STATES, NUM_ACTIONS))
    R = init_reward_function()
    policy = np.full(NUM_STATES, '', dtype=object)
    acc_rewards = [0]

    # Repeating for each episode
    for _ in range(num_episodes):
        curr_state_idx = get_random_state_idx()
        episode_length = 0
        ep_acc_reward = 0

        # Repeating for each step of the episode
        while episode_length < max_episode_length:
            action_idx = epsilon_greedy(Q, curr_state_idx, epsilon)
            action = f"a{action_idx+1}"
            next_state_idx = compute_next_state_idx(p, curr_state_idx, action_idx)
            reward = R[action][curr_state_idx, next_state_idx]
            ep_acc_reward += reward
            curr_Q = Q[curr_state_idx, action_idx]

            Q[curr_state_idx, action_idx] = curr_Q + alpha * (reward + gamma * np.max(Q[next_state_idx]) - curr_Q)
            policy[curr_state_idx] = f"a{np.argmax(Q[curr_state_idx])+1}"

            curr_state_idx = next_state_idx
            episode_length += 1
        
        avg_ep_acc_reward = ep_acc_reward / episode_length
        acc_rewards.append(avg_ep_acc_reward)

    return policy, Q, acc_rewards

def sarsa(p, gamma, alpha, epsilon, num_episodes, max_episode_length):
    """
    Function to perform SARSA
    """
    Q = np.zeros((NUM_STATES, NUM_ACTIONS))
    R = init_reward_function()
    policy = np.full(NUM_STATES, '', dtype=object)
    acc_rewards = [0]

    # Repeating for each episode
    for _ in range(num_episodes):
        curr_state_idx = get_random_state_idx()
        curr_action_idx = epsilon_greedy(Q, curr_state_idx, epsilon)
        episode_length = 0
        ep_acc_reward = 0

        # Repeating for each step of the episode
        while episode_length < max_episode_length:
            next_state_idx = compute_next_state_idx(p, curr_state_idx, curr_action_idx)
            curr_action = f"a{curr_action_idx+1}"
            reward = R[curr_action][curr_state_idx, next_state_idx]

            curr_Q = Q[curr_state_idx, curr_action_idx]
            ep_acc_reward += reward

            next_action_idx = epsilon_greedy(Q, next_state_idx, epsilon)
            next_Q = Q[next_state_idx, next_action_idx]

            Q[curr_state_idx, curr_action_idx] = curr_Q + alpha * (reward + gamma * next_Q - curr_Q)
            policy[curr_state_idx] = f"a{np.argmax(Q[curr_state_idx])+1}"

            curr_state_idx = next_state_idx
            curr_action_idx = next_action_idx
            episode_length += 1
        
        avg_ep_acc_reward = ep_acc_reward / episode_length
        acc_rewards.append(avg_ep_acc_reward)

    return policy, Q, acc_rewards

def actor_critic(p, gamma, alpha, beta, num_episodes, max_episode_length):
    """
    Function to perform Actor-Critic method
    """
    V = np.zeros((NUM_STATES))
    H = np.zeros((NUM_STATES, NUM_ACTIONS))
    R = init_reward_function()
    policy = np.full(NUM_STATES, '', dtype=object)
    acc_rewards = [0]

    # Repeating for each episode
    for _ in range(num_episodes):
        curr_state_idx = get_random_state_idx()
        episode_length = 0
        ep_acc_reward = 0

        # Repeating for each step of the episode
        while episode_length < max_episode_length:
            preferences = get_preference(H, curr_state_idx)
            curr_action_idx = np.random.choice(range(NUM_ACTIONS), p=preferences)
            curr_action = f"a{curr_action_idx+1}"
            next_state_idx = compute_next_state_idx(p, curr_state_idx, curr_action_idx)
            reward = R[curr_action][curr_state_idx, next_state_idx]
            ep_acc_reward += reward

            curr_H = H[curr_state_idx, curr_action_idx]
            curr_action_preference = get_preference(H, curr_state_idx, curr_action_idx)

            delta = reward + gamma * V[next_state_idx] - V[curr_state_idx]
            V[curr_state_idx] = V[curr_state_idx] + alpha * delta
            H[curr_state_idx, curr_action_idx] = curr_H + beta * delta * (1 - curr_action_preference)
            policy[curr_state_idx] = f"a{np.argmax(H[curr_state_idx])+1}"

            curr_state_idx = next_state_idx
            episode_length += 1
        
        avg_ep_acc_reward = ep_acc_reward / episode_length
        acc_rewards.append(avg_ep_acc_reward)

    return policy, H, acc_rewards

def sarsa_lambda(p, gamma, alpha, epsilon, num_episodes, max_episode_length, lambda_val):
    """
    Function to perform SARSA(lambda)
    """
    Q = np.zeros((NUM_STATES, NUM_ACTIONS))
    R = init_reward_function()
    policy = np.full(NUM_STATES, '', dtype=object)
    acc_rewards = [0]

    # Repeating for each episode
    for _ in range(num_episodes):
        E = np.zeros((NUM_STATES, NUM_ACTIONS))
        curr_state_idx = get_random_state_idx()
        curr_action_idx = epsilon_greedy(Q, curr_state_idx, epsilon)
        episode_length = 0
        ep_acc_reward = 0

        # Repeating for each step of the episode
        while episode_length < max_episode_length:
            next_state_idx = compute_next_state_idx(p, curr_state_idx, curr_action_idx)
            curr_action = f"a{curr_action_idx+1}"
            reward = R[curr_action][curr_state_idx, next_state_idx]
            ep_acc_reward += reward
            next_action_idx = epsilon_greedy(Q, next_state_idx, epsilon)

            delta = reward + gamma * Q[next_state_idx, next_action_idx] - Q[curr_state_idx, curr_action_idx]
            E[curr_state_idx, curr_action_idx] += 1
            Q = Q + alpha * delta * E
            E = gamma * lambda_val * E
            policy[curr_state_idx] = f"a{np.argmax(Q[curr_state_idx])+1}"

            curr_state_idx = next_state_idx
            curr_action_idx = next_action_idx
            episode_length += 1
        
        avg_ep_acc_reward = ep_acc_reward / episode_length
        acc_rewards.append(avg_ep_acc_reward)

    return policy, Q, acc_rewards

def simulate(algo_name, num_runs, p, gamma, alpha, beta, epsilon, lambda_val, num_episodes, max_episode_length):
    """
    Function to simulate TD algorithms for multiple runs
    """
    run_acc_rewards = []
    for i in range(num_runs):

        if algo_name == "Q-Learning":
            policy, Q, acc_rewards = q_learning(p, gamma, alpha, epsilon, num_episodes, max_episode_length)

        elif algo_name == "SARSA":
            policy, Q, acc_rewards = sarsa(p, gamma, alpha, epsilon, num_episodes, max_episode_length)
        
        elif algo_name == "Actor_Critic":
            policy, H, acc_rewards = actor_critic(p, gamma, alpha, beta, num_episodes, max_episode_length)

        elif algo_name == "sarsa_lambda":
            policy, H, acc_rewards = sarsa_lambda(p, gamma, alpha, epsilon, num_episodes, max_episode_length, lambda_val)

        run_acc_rewards.append(acc_rewards)
        
        print(policy)
        print(f"Run {i+1} completed.")

    avg_acc_rewards = calculate_avg_acc_rewards(run_acc_rewards, num_episodes, num_runs)
    
    return avg_acc_rewards

def simulate_all(num_runs, p, gamma, alpha, beta, epsilon, lambda_val, num_episodes, max_episode_length):
    algos = ["Q-Learning", "SARSA", "Actor_Critic", "sarsa_lambda"]
    avg_acc_rewards = {algo: None for algo in algos}

    for algo in algos:
        print(algo)
        avg_acc_rewards[algo] = simulate(algo, num_runs, p, gamma, alpha, beta, epsilon, lambda_val, num_episodes, max_episode_length)
    
    plot_avg_acc_rewards(avg_acc_rewards, f"Average Accumulated Rewards")
    
    return avg_acc_rewards


In [ ]:
p = 0.1
gamma = 0.9
alpha = 0.25
beta = 0.05
lambda_val = 0.95
epsilon = 0.15
max_episode_length = 100
num_episodes = 1000
num_runs = 10

avg_acc_rewards = simulate_all(num_runs, p, gamma, alpha, beta, epsilon, lambda_val, num_episodes, max_episode_length)
